In [1]:
"""This code calculated to use a high res. topo tiff, setup matfile output and still water levels + SLR
to get flood depths in a tiff. Setup matfile output is interpolated on to the topo tiff to get total flood depths.
flood depth = Total water level + wave setup - topo 
Written by Taran Kalra, July 22, 2024"""


'This code calculated to use a high res. topo tiff, setup matfile output and still water levels + SLR\nto get flood depths in a tiff. Setup matfile output is interpolated on to the topo tiff to get total flood depths.\nWritten by Taran Kalra, July 22, 2024'

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat              
import rioxarray
import rasterio 
from create_raster import make_swan_raster_StructuredGrid

In [3]:
# Grid specific inputs
PR = 0 
St_Croix = 1
if(St_Croix):
    gridsize = 10  # in meter
    gridx = 483 #j direction + 1 from swan file 
    gridy = 278 #i direction + 1 from swan file 
    xMin = 389822 #this is your x origin in state plane m (or whatever you used)
    yMin = 192378 #this is your y origin in state plane m (or whatever you used)
            
    EPSG = 32163 # EPSGS correspondign to USVI (ST.Croix )
    angle_degrees = 0.0  # Angle of the grid
if(PR):
    gridsize = 5  # in meter
    gridx = 1095 # direction + 1 from swan file 
    gridy = 1048 #i direction + 1 from swan file 
    xMin = 316520 #this is your x origin in state plane m (or whatever you used)
    yMin = 247622 #this is your y origin in state plane m (or whatever you used)
            
    EPSG = 32161 # EPSGS correspondign to PR
    angle_degrees = 41.6  # Angle of the grid
    
m_to_feet = 3.28084
varname = "Setup"

In [4]:
cases = ["1perc", "2perc", "5perc", "10perc", "20perc", "50perc"]
TWL = [    1.8,    1.71,    1.62,     1.5,       1.4,    1.32 ]
scene = "Degraded" # "Restored"

if (scene == "Restored"):
    scenario = "Restored" # "Degraded"
    scenario1= "restored" # degraded"  
    scenario_out = "alt"  # "no_alt"
    gridfile = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Grid\restored_USVI_rasterio.tif'

else:
    scenario = "Degraded"
    scenario1 = "degraded"
    scenario_out = "no_alt"
    gridfile = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Grid\degraded_USVI_rasterio.tif'

# input file names
input_directory = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_Runs'
output_directory = r'C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_final\Flood_depth'

In [5]:
# Loop over the filenames in the directory
for i, case in enumerate(cases):
    matfile = f'{input_directory}\{scenario}_Reef\{case}\{case}_{scenario1}_outs.mat'
    outfile = f'{output_directory}\{case}_{scenario_out}.tif' 

    print(matfile)
    print(outfile) 
    
# Temporary filenames   
    outfile_coarse_raster = 'tiffs_back\output_raster_coarse_setup.tif'
    outfile_refine_raster = 'tiffs_back\output_raster_refine_setup.tif'    
    
    # Get the Hsig 
    make_swan_raster_StructuredGrid(outfile_coarse_raster, matfile, varname, EPSG, xMin, yMin, gridx, gridy, gridsize, angle_degrees)

    # Grid file path specified 
    # this is the finer TIFF file 
    finer_xds = rioxarray.open_rasterio(gridfile) 
    topobathy = finer_xds.squeeze()
    # Curate topbthy 
    topobathy = np.where(topobathy<-9999, np.nan, topobathy)

    # Open this newly created tiff on the SWAN grid 
    coarse_data = rioxarray.open_rasterio(outfile_coarse_raster)    
    # Interpolate wave setup from SWAN grid to the topobathy tiff (merged tiff)
    ds_new = coarse_data.interp(y=finer_xds.y, x=finer_xds.x, method='nearest') # finer_xds that is on refined SWAN grid 
    ds_new = ds_new.squeeze() #This is now wave setup interpolated over the SWAN grid that is refined 

    # Fill the array to get flood depth, note TWL is variable for all scenarios
    # ds_new is the wave setup interpolated the SWAN grid that is refined 
    ds_new = ds_new + TWL[i] - topobathy # topobathy is in meter from NOAA has reverse sign (ocean is negative, land is positive) 
    #ds_new = (ds_new + TWL[i])*0.0 - topobathy
 
    print("Converting meters to feet")
    #ds_new = ds_new.squeeze()
    ds_new = m_to_feet*ds_new # (ds_new is in feet)
    print(type(ds_new))
    ds_new.rio.to_raster(outfile_refine_raster) # save the new array in a raster before changing the profile of the raster 

    # Grid file used to copy the profile 
    with rasterio.open(gridfile, 'r') as src:
        profile1 = src.profile.copy()

    #   the profile in the output raster1 and then add feet in units 
    with rasterio.open(outfile_refine_raster) as src: 
        profile = src.profile
        profile = profile1 
        array = src.read(1)
    # Update the profile to change the units from meters to feet    
    profile['units'] = 'Feet'

    # Save the updated array to a new TIFF file
    with rasterio.open(outfile, 'w+', **profile) as dst:
        dst.write(ds_new, 1)
        dst.update_tags(units=profile['units'])

    del outfile_refine_raster, outfile_coarse_raster

C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_Runs\Degraded_Reef\1perc\1perc_degraded_outs.mat
C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_final\Flood_depth\1perc_no_alt.tif
raster finished
Converting meters to feet
<class 'xarray.core.dataarray.DataArray'>
C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_Runs\Degraded_Reef\2perc\2perc_degraded_outs.mat
C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_final\Flood_depth\2perc_no_alt.tif
raster finished
Converting meters to feet
<class 'xarray.core.dataarray.DataArray'>
C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_Runs\Degraded_Reef\5perc\5perc_degraded_outs.mat
C:\Users\tkalra\Desktop\NFWP_Swan_coral_USVirgin\69532_Coral_Reefs_St_Croix\Swan_final\Flood_depth\5perc_no_alt.tif
raster finished
Converting meters to feet
<class 'xarray.core.dataarray.DataArray'>
C:\Users\t

In [6]:
print(np.nanmin(ds_new))
print(np.nanmax(ds_new))

-74.83735
64.71292


In [7]:

#fig = plt.figure(figsize=(15,10))
#ax = fig.add_subplot(1,1,1)
#z1_plot = ax.pcolormesh(fine_data.x.values, fine_data.y.values, array, cmap='viridis', vmin=-20, vmax=20)
#plt.colorbar(z1_plot)